In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

plt.rcParams['figure.figsize'] = (10, 6)


In [2]:
municipios = pd.read_csv(
    "../data/raw/Pesquisa Agrícola Municipal (PAM)/6. caju_castanha_piaui_municipios.csv"
)
municipios.head()

,ano,id_municipio,nome_municipio,produto,area_colhida_ha,producao_total,produtividade_media,valor_total_producao
0,1974,2200103,Agricolândia,Caju,15,780.0,52000.0,0.0
1,1974,2200400,Altos,Caju,180,4500.0,25000.0,0.0
2,1974,2200509,Amarante,Caju,38,1900.0,50000.0,0.0
3,1974,2200608,Angical do Piauí,Caju,8,400.0,50000.0,0.0
4,1974,2200905,Aroazes,Caju,12,960.0,80000.0,0.0


In [ ]:
# Análise exploratória dos dados municipais
municipios_limpo = municipios.dropna(
    subset=["producao_total", "valor_total_producao"]
)

municipios_limpo = municipios_limpo[
    municipios_limpo["valor_total_producao"] > 0
]
municipios_limpo["produto"].value_counts()
municipios_limpo["ano"].min(), municipios_limpo["ano"].max()



(1986, 2024)

In [ ]:
# Ranking dos municípios produtores de castanha de caju
ranking_municipios = (
    municipios_limpo
    .groupby(["id_municipio", "nome_municipio"], as_index=False)
    .agg({
        "producao_total": "sum",
        "valor_total_producao": "sum",
        "area_colhida_ha": "sum"
    })
)


In [ ]:
# Ranking dos municípios por produção total
ranking_producao = ranking_municipios.sort_values(
    by="producao_total",
    ascending=False
)

ranking_producao.head(10)


,id_municipio,nome_municipio,producao_total,valor_total_producao,area_colhida_ha
151,2208205,Pio IX,4499739.0,287298.5852,1267205
52,2202703,Cocal,54510.0,51222.2176,216745
193,2210607,São Raimundo Nonato,44693.0,22779.4488,341662
169,2209401,Santo Antônio de Lisboa,33356.0,39122.3698,138376
123,2206506,Monsenhor Hipólito,28792.0,42916.6256,160030
80,2204204,Francisco Santos,27033.0,32065.7024,155944
54,2202729,Cocal dos Alves,26606.0,44977.0000,94407
94,2205201,Jaicós,23620.0,24219.0640,107873
2,2200251,Alagoinha do Piauí,21669.0,24099.2385,141842
44,2202307,Canto do Buriti,21603.0,11540.9104,110462


In [ ]:
# Ranking dos municípios por valor total da produção
ranking_valor = ranking_municipios.sort_values(
    by="valor_total_producao",
    ascending=False
)

ranking_valor.head(10)


,id_municipio,nome_municipio,producao_total,valor_total_producao,area_colhida_ha
151,2208205,Pio IX,4499739.0,287298.5852,1267205
52,2202703,Cocal,54510.0,51222.2176,216745
54,2202729,Cocal dos Alves,26606.0,44977.0000,94407
123,2206506,Monsenhor Hipólito,28792.0,42916.6256,160030
169,2209401,Santo Antônio de Lisboa,33356.0,39122.3698,138376
80,2204204,Francisco Santos,27033.0,32065.7024,155944
88,2204709,Inhuma,17953.0,28562.9632,72956
94,2205201,Jaicós,23620.0,24219.0640,107873
2,2200251,Alagoinha do Piauí,21669.0,24099.2385,141842
193,2210607,São Raimundo Nonato,44693.0,22779.4488,341662


In [ ]:
# Especialização produtiva por município
especializacao = (
    municipios_limpo
    .groupby(["id_municipio", "nome_municipio"], as_index=False)
    .agg(
        anos_com_producao=("ano", "nunique"),
        producao_total=("producao_total", "sum"),
        valor_total_producao=("valor_total_producao", "sum")
    )
)
especializacao.head(10)



,id_municipio,nome_municipio,anos_com_producao,producao_total,valor_total_producao
0,2200103,Agricolândia,33,951.0,1494.4254
1,2200202,Água Branca,35,788.0,711.6205
2,2200251,Alagoinha do Piauí,36,21669.0,24099.2385
3,2200277,Alegrete do Piauí,32,2027.0,1954.0571
4,2200301,Alto Longá,35,2846.0,3583.1929
5,2200400,Altos,35,6519.0,5432.8850
6,2200459,Alvorada do Gurguéia,28,1023.0,1116.0000
7,2200509,Amarante,35,4139.0,5391.8318
8,2200608,Angical do Piauí,29,817.0,667.1350
9,2200707,Anísio de Abreu,24,131.0,153.0000


In [ ]:
# Cálculo do percentual de anos com produção
total_anos = municipios_limpo["ano"].nunique()

especializacao["percentual_anos"] = (
    especializacao["anos_com_producao"] / total_anos
)

def classificar_especializacao(p):
    if p >= 0.75:
        return "Alta especialização"
    elif p >= 0.40:
        return "Especialização média"
    else:
        return "Baixa especialização"

especializacao["grau_especializacao"] = (
    especializacao["percentual_anos"]
    .apply(classificar_especializacao)
)


especializacao["grau_especializacao"].value_counts()

especializacao_final = especializacao.sort_values(
    by=["grau_especializacao", "valor_total_producao"],
    ascending=[True, False]
)

especializacao_final.head(10)


,id_municipio,nome_municipio,anos_com_producao,producao_total,valor_total_producao,percentual_anos,grau_especializacao
151,2208205,Pio IX,39,4499739.0,287298.5852,1.000000,Alta especialização
52,2202703,Cocal,37,54510.0,51222.2176,0.948718,Alta especialização
123,2206506,Monsenhor Hipólito,36,28792.0,42916.6256,0.923077,Alta especialização
169,2209401,Santo Antônio de Lisboa,36,33356.0,39122.3698,0.923077,Alta especialização
80,2204204,Francisco Santos,36,27033.0,32065.7024,0.923077,Alta especialização
88,2204709,Inhuma,35,17953.0,28562.9632,0.897436,Alta especialização
94,2205201,Jaicós,36,23620.0,24219.0640,0.923077,Alta especialização
2,2200251,Alagoinha do Piauí,36,21669.0,24099.2385,0.923077,Alta especialização
193,2210607,São Raimundo Nonato,37,44693.0,22779.4488,0.948718,Alta especialização
68,2203404,Dom Expedito Lopes,35,16061.0,20681.2300,0.897436,Alta especialização


# ANÁLISE DE CONCENTRAÇÃO PRODUTIVA MUNICIPAL

Aqui queremos responder:

Poucos municípios concentram a maior parte da produção e do valor?

In [ ]:
# Participação percentual por município
concentracao = ranking_municipios.copy()

concentracao["participacao_producao"] = (
    concentracao["producao_total"] /
    concentracao["producao_total"].sum()
)

concentracao["participacao_valor"] = (
    concentracao["valor_total_producao"] /
    concentracao["valor_total_producao"].sum()
)

In [10]:
concentracao.sort_values(
    by="participacao_valor",
    ascending=False
).head(10)


,id_municipio,nome_municipio,producao_total,valor_total_producao,area_colhida_ha,participacao_producao,participacao_valor
151,2208205,Pio IX,4499739.0,287298.5852,1267205,0.860224,0.271343
52,2202703,Cocal,54510.0,51222.2176,216745,0.010421,0.048378
54,2202729,Cocal dos Alves,26606.0,44977.0000,94407,0.005086,0.042479
123,2206506,Monsenhor Hipólito,28792.0,42916.6256,160030,0.005504,0.040533
169,2209401,Santo Antônio de Lisboa,33356.0,39122.3698,138376,0.006377,0.036950
80,2204204,Francisco Santos,27033.0,32065.7024,155944,0.005168,0.030285
88,2204709,Inhuma,17953.0,28562.9632,72956,0.003432,0.026977
94,2205201,Jaicós,23620.0,24219.0640,107873,0.004515,0.022874
2,2200251,Alagoinha do Piauí,21669.0,24099.2385,141842,0.004143,0.022761
193,2210607,São Raimundo Nonato,44693.0,22779.4488,341662,0.008544,0.021514


# EFICIÊNCIA ECONÔMICA MUNICIPAL

Agora vamos responder:

Quem transforma produção em valor de forma mais eficiente?

In [14]:
# Eficiência produtiva por município
eficiencia_municipal = ranking_municipios.copy()

eficiencia_municipal["valor_por_unidade"] = (
    eficiencia_municipal["valor_total_producao"] /
    eficiencia_municipal["producao_total"]
)
eficiencia_municipal.sort_values(
    by="valor_por_unidade",
    ascending=False
).head(5)


,id_municipio,nome_municipio,producao_total,valor_total_producao,area_colhida_ha,valor_por_unidade
128,2206720,Nazária,315.0,688.0,1549,2.184127
82,2204352,Geminiano,6248.0,13342.0,30122,2.135403
30,2201945,Boqueirão do Piauí,524.0,1110.0,1890,2.118321
181,2209971,São João do Arraial,121.0,238.0,396,1.966942
126,2206696,Murici dos Portelas,1264.0,2444.0,4288,1.933544


# VISUALIZAÇÕES (GRÁFICOS)

Agora transformamos isso em leitura visual clara.

In [ ]:
# Visualização da concentração do valor da produção por município
top_valor = concentracao.sort_values(
    by="participacao_valor",
    ascending=False
).head(10)

fig = px.bar(
    top_valor,
    x="nome_municipio",
    y="participacao_valor",
    title="Concentração do Valor da Cajucultura — Top 10 Municípios (Piauí)",
    labels={
        "nome_municipio": "Município",
        "participacao_valor": "Participação no valor total"
    }
)

fig.update_layout(
    template="plotly_dark",
    title_x=0.5
)

fig.show()


In [ ]:
# Visualização da eficiência econômica por município
top_eficiencia = eficiencia_municipal.sort_values(
    by="valor_por_unidade",
    ascending=False
).head(10)

fig = px.bar(
    top_eficiencia,
    x="nome_municipio",
    y="valor_por_unidade",
    title="Eficiência Econômica Municipal — Top 10 (Piauí)",
    labels={
        "nome_municipio": "Município",
        "valor_por_unidade": "Valor por unidade produzida"
    }
)

fig.update_layout(
    template="plotly_dark",
    title_x=0.5
)

fig.show()


# Produção × Valor (Dispersão Municipal)

Quais Municípios que produzem mais são os que mais geram valor?

In [17]:
# Visualização da relação entre produção física e valor da produção por município
fig = px.scatter(
    ranking_municipios,
    x="producao_total",
    y="valor_total_producao",
    hover_name="nome_municipio",
    title="Produção Física × Valor da Produção — Municípios do Piauí",
    labels={
        "producao_total": "Produção total",
        "valor_total_producao": "Valor total da produção (R$)"
    }
)

fig.update_layout(
    template="plotly_dark",
    title_x=0.5
)

fig.show()


# Persistência Temporal Municipal
Quais municípios sustentam a cajucultura ao longo do tempo?

In [18]:
# Visualização da persistência temporal da cajucultura nos municípios do Piauí
fig = px.histogram(
    especializacao,
    x="anos_com_producao",
    nbins=20,
    title="Persistência Temporal da Cajucultura nos Municípios do Piauí",
    labels={
        "anos_com_producao": "Número de anos com produção registrada",
        "count": "Número de municípios"
    }
)

fig.update_layout(
    template="plotly_dark",
    title_x=0.5
)

fig.show()


# Tipologia Municipal (4 Quadrantes)

Que tipos de municípios existem na cajucultura do Piauí?

In [21]:
mediana_producao = ranking_municipios["producao_total"].median()
mediana_valor = ranking_municipios["valor_total_producao"].median()


In [22]:
def classificar_tipologia(row):
    if row["producao_total"] >= mediana_producao and row["valor_total_producao"] >= mediana_valor:
        return "Alto volume / Alto valor"
    elif row["producao_total"] >= mediana_producao and row["valor_total_producao"] < mediana_valor:
        return "Alto volume / Baixo valor"
    elif row["producao_total"] < mediana_producao and row["valor_total_producao"] >= mediana_valor:
        return "Baixo volume / Alto valor"
    else:
        return "Baixo volume / Baixo valor"

ranking_municipios["tipologia"] = ranking_municipios.apply(classificar_tipologia, axis=1)


In [23]:
fig = px.scatter(
    ranking_municipios,
    x="producao_total",
    y="valor_total_producao",
    color="tipologia",
    hover_name="nome_municipio",
    title="Tipologia Municipal da Cajucultura no Piauí",
    labels={
        "producao_total": "Produção total",
        "valor_total_producao": "Valor total da produção (R$)"
    }
)

fig.update_layout(
    template="plotly_dark",
    title_x=0.5
)

fig.show()
